<a href="https://colab.research.google.com/github/imammarzuki/Eksperimen/blob/main/Hybrid_CF_KNN_dan_CBF_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357239 sha256=0e188c6454469ef432344990ec07b98f45bc22f362629370349b4960e3872798
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_absolute_error, mean_squared_error, confusion_matrix
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
place=pd.read_csv("drive/MyDrive/S3/Eksperimen/Data/tourism_with_id.csv")
rating=pd.read_csv("drive/MyDrive/S3/Eksperimen/Data/tourism_rating.csv")
user=pd.read_csv("drive/MyDrive/S3/Eksperimen/Data/user.csv")

In [ ]:
place.columns

Index(['Place_Id', 'Place_Name', 'Description', 'Category', 'City', 'Price',
       'Rating', 'Time_Minutes', 'Coordinate', 'Lat', 'Long', 'Unnamed: 11',
       'Unnamed: 12'],
      dtype='object')

In [ ]:
place1=place[["Place_Id", "Place_Name", "Description", "City", "Price", "Rating"]]

In [ ]:
place1.columns

Index(['Place_Id', 'Place_Name', 'Description', 'City', 'Price', 'Rating'], dtype='object')

In [ ]:
place2=place1[place1["City"]=="Surabaya"]
place2

,Place_Id,Place_Name,Description,City,Price,Rating
391,392,Ekowisata Mangrove Wonorejo,Hutan Wisata Mangrove Surabaya merupakan wisat...,Surabaya,0,4.3
392,393,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,Surabaya,0,4.4
393,394,Air Mancur Menari,Jembatan Kenjeran dengan air mancur menarinya ...,Surabaya,35000,4.4
394,395,Taman Prestasi,Taman Prestasi Surabaya merupakan salah satu t...,Surabaya,0,4.6
395,396,Monumen Kapal Selam,"Monumen Kapal Selam, atau disingkat Monkasel, ...",Surabaya,15000,4.4
396,397,Taman Kunang-Kunang,Taman Kunang – Kunang di Surabaya ini mempunya...,Surabaya,0,4.4
397,398,Taman Buah Surabaya,Wisata Taman Buah Undaan di Surabaya adalah sa...,Surabaya,0,4.1
398,399,Taman Pelangi,Kalau pelangi biasanya ada di siang hari pasca...,Surabaya,0,4.5
399,400,Hutan Bambu Keputih,Surabaya menjadi kota metropolitan kedua setel...,Surabaya,0,4.3
400,401,Taman Keputran,"Ntah, mengapa nama taman ini disebut dengan ta...",Surabaya,0,4.3


In [ ]:
place2=place2.reset_index(drop=True)
place2

,Place_Id,Place_Name,Description,City,Price,Rating
0,392,Ekowisata Mangrove Wonorejo,Hutan Wisata Mangrove Surabaya merupakan wisat...,Surabaya,0,4.3
1,393,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,Surabaya,0,4.4
2,394,Air Mancur Menari,Jembatan Kenjeran dengan air mancur menarinya ...,Surabaya,35000,4.4
3,395,Taman Prestasi,Taman Prestasi Surabaya merupakan salah satu t...,Surabaya,0,4.6
4,396,Monumen Kapal Selam,"Monumen Kapal Selam, atau disingkat Monkasel, ...",Surabaya,15000,4.4
5,397,Taman Kunang-Kunang,Taman Kunang – Kunang di Surabaya ini mempunya...,Surabaya,0,4.4
6,398,Taman Buah Surabaya,Wisata Taman Buah Undaan di Surabaya adalah sa...,Surabaya,0,4.1
7,399,Taman Pelangi,Kalau pelangi biasanya ada di siang hari pasca...,Surabaya,0,4.5
8,400,Hutan Bambu Keputih,Surabaya menjadi kota metropolitan kedua setel...,Surabaya,0,4.3
9,401,Taman Keputran,"Ntah, mengapa nama taman ini disebut dengan ta...",Surabaya,0,4.3


In [ ]:
place2.columns

Index(['Place_Id', 'Place_Name', 'Description', 'City', 'Price', 'Rating'], dtype='object')

In [ ]:
rating.columns

Index(['User_Id', 'Place_Id', 'Place_Ratings'], dtype='object')

In [ ]:
user.columns

Index(['User_Id', 'Location', 'Age'], dtype='object')

In [ ]:
data1=pd.merge(rating, place2, on="Place_Id")
data1.head()

,User_Id,Place_Id,Place_Ratings,Place_Name,Description,City,Price,Rating
0,1,393,5,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,Surabaya,0,4.4
1,6,393,2,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,Surabaya,0,4.4
2,8,393,5,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,Surabaya,0,4.4
3,19,393,3,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,Surabaya,0,4.4
4,27,393,3,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,Surabaya,0,4.4


In [ ]:
data2=data1.rename({"User_Id":"user_id", "Place_Id":"place_id",  "Place_Ratings":"rating1", "Place_Name":"place_name", "Description":"description","Price":"price", "Rating":"rating2"}, axis=1)

In [ ]:
data2.columns

Index(['user_id', 'place_id', 'rating1', 'place_name', 'description', 'City',
       'price', 'rating2'],
      dtype='object')

In [ ]:
df=data2[["user_id","place_id", "place_name", "description", "price", "rating1", "rating2"]]

In [ ]:
df.head()

,user_id,place_id,place_name,description,price,rating1,rating2
0,1,393,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,0,5,4.4
1,6,393,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,0,2,4.4
2,8,393,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,0,5,4.4
3,19,393,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,0,3,4.4
4,27,393,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,0,3,4.4


In [ ]:
# Menggabungkan rating menjadi satu nilai rata-rata
df['average_rating'] = df[['rating1', 'rating2']].mean(axis=1)

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['description'])

# Menghitung cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
# Data untuk CF
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df[['user_id', 'place_id', 'average_rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)

# Menggunakan KNN
algo = KNNBasic(sim_options={'name': 'cosine', 'user_based': True})
algo.fit(trainset)
predictions = algo.test(testset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
# Prediksi CF
cf_predictions = np.array([pred.est for pred in predictions])

# Gabungkan dengan CBF
final_predictions = []
for pred in predictions:
    place_idx = df[df['place_id'] == pred.iid].index[0]
    cbf_score = np.mean(cosine_sim[place_idx])
    final_score = (cbf_score + pred.est) / 2
    final_predictions.append(final_score)

# Actual ratings
actual_ratings = np.array([pred.r_ui for pred in predictions])

# Hitung metrik
mae = mean_absolute_error(actual_ratings, final_predictions)
rmse = np.sqrt(mean_squared_error(actual_ratings, final_predictions))
cm = confusion_matrix(actual_ratings.round(), np.array(final_predictions).round())

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print("Confusion Matrix:\n", cm)

MAE: 1.8232905599999851
RMSE: 1.9607677308727087
Confusion Matrix:
 [[  0   0   0   0   0]
 [  0   0   0   0   0]
 [  0 102   0   0   0]
 [  3 101   0   0   0]
 [  3  54   0   0   0]]
